In [173]:
import os
import boto3
import sys
from prophet import Prophet
import matplotlib.pyplot as plt
import datetime
import pandas as pd



In [174]:
instance_name = 't3.medium'

In [175]:
results = []

def history(client, instance, region_):
    INSTANCE = instance
    STARTTIME = (datetime.datetime.now() - datetime.timedelta(days=90)).isoformat()
    
    prices = client.describe_spot_price_history(
        InstanceTypes=[INSTANCE],
        ProductDescriptions=['Linux/UNIX'],
        AvailabilityZone="{region_}a".format(region_= region_),
        StartTime=STARTTIME,
        MaxResults=200
    )
    # print("Prices for region %s: %s" % (region, prices["SpotPriceHistory"]))
    for price in prices["SpotPriceHistory"]:
        results.append({
                        'Timestamp': price["Timestamp"].strftime('%m-%d-%Y'), 
                        'Price': price["SpotPrice"]
                        })

        
    df = pd.DataFrame.from_dict(results)
    df = df.rename(columns={'Timestamp': 'ds', 'Price': 'y'})
    df = df.sort_values('y', ascending=False).drop_duplicates('ds').sort_values('ds').reset_index(drop=True)
    
    return df

In [176]:
def interpolate(df):
    # Split the data into train and test sets
    # Train the model on the train set
    model = Prophet(
        daily_seasonality=True,
        yearly_seasonality=False,
    )
    model.fit(df)

    # Predict for the test set
    future = model.make_future_dataframe(periods=1)
    forecast = model.predict(future)
    predicted_price = forecast['yhat'].iloc[-1]
    return predicted_price

In [177]:
def main(instance):
    span = 30
    region = "us-east-1"
    client = boto3.client('ec2', region_name=region)
    df = history(client, instance, region)
    print(len(df))
    bid_price = 0
    for i in range(len(df) - span):
        print(f'Itertaion: {i}')
        new_df = df.iloc[i:i+span]
        print(f' new df start: {new_df.iloc[0]}')
        print(f' new df end: {new_df.iloc[-1]}')
        price = interpolate(new_df)
        df.loc[i+span, 'yhat'] = round(price + price*0.05, 4)
        if(i == 0 or bid_price <= float(df.loc[i+span, 'y'])):
            bid_price = round(price + price*0.05, 4)
        df.loc[i+span, 'bidding_price'] = bid_price
    print(df)
    return df

In [178]:
# spots = {"t2.small": {"cpu": 1, "memory": 2, "date": "2023-03-22", "cost": 0.008}, "t3.medium": {"cpu": 2, "memory": 4, "date": "2023-03-19", "cost": 0.019}, "m6g.medium": {"cpu": 1, "memory": 4, "date": 22, "cost": 0.01}, "c6a.large": {"cpu": 2, "memory": 4, "date": 22, "cost": 0.01}, "t4g.large": {"cpu": 2, "memory": 8, "date": 22, "cost": 0.01}, "c6g.xlarge": {"cpu": 4, "memory": 8, "date": 22, "cost": 0.01}}
# instances = list(spots.keys())
# print(instances)
# for i in instances:
#   print(f'Instance: {i}')
#   main(i)

updated_df = main(instance_name)


INFO:prophet:n_changepoints greater than number of observations. Using 23.


71
Itertaion: 0
 new df start: ds    04-01-2023
y       0.027900
Name: 0, dtype: object
 new df end: ds    04-30-2023
y       0.035400
Name: 29, dtype: object


INFO:prophet:n_changepoints greater than number of observations. Using 23.


Itertaion: 1
 new df start: ds               04-02-2023
y                  0.029100
yhat                    NaN
bidding_price           NaN
Name: 1, dtype: object
 new df end: ds               05-01-2023
y                  0.035000
yhat                 0.0374
bidding_price        0.0374
Name: 30, dtype: object


INFO:prophet:n_changepoints greater than number of observations. Using 23.


...

Itertaion: 38
 new df start: ds               05-09-2023
y                  0.035700
yhat                  0.037
bidding_price        0.0374
Name: 38, dtype: object
 new df end: ds               06-07-2023
y                  0.036600
yhat                 0.0385
bidding_price        0.0374
Name: 67, dtype: object


INFO:prophet:n_changepoints greater than number of observations. Using 23.


Itertaion: 39
 new df start: ds               05-10-2023
y                  0.035700
yhat                  0.037
bidding_price        0.0374
Name: 39, dtype: object
 new df end: ds               06-08-2023
y                  0.036500
yhat                 0.0386
bidding_price        0.0374
Name: 68, dtype: object


INFO:prophet:n_changepoints greater than number of observations. Using 23.


Itertaion: 40
 new df start: ds               05-11-2023
y                  0.035700
yhat                 0.0374
bidding_price        0.0374
Name: 40, dtype: object
 new df end: ds               06-09-2023
y                  0.035800
yhat                 0.0384
bidding_price        0.0374
Name: 69, dtype: object
            ds         y    yhat  bidding_price
0   04-01-2023  0.027900     NaN            NaN
1   04-02-2023  0.029100     NaN            NaN
2   04-03-2023  0.031100     NaN            NaN
3   04-04-2023  0.032400     NaN            NaN
4   04-05-2023  0.032900     NaN            NaN
..         ...       ...     ...            ...
66  06-06-2023  0.036500  0.0388         0.0374
67  06-07-2023  0.036600  0.0385         0.0374
68  06-08-2023  0.036500  0.0386         0.0374
69  06-09-2023  0.035800  0.0384         0.0374
70  06-10-2023  0.035400  0.0380         0.0374

[71 rows x 4 columns]


In [179]:
updated_df

,ds,y,yhat,bidding_price
0,04-01-2023,0.027900,NaN,NaN
1,04-02-2023,0.029100,NaN,NaN
2,04-03-2023,0.031100,NaN,NaN
3,04-04-2023,0.032400,NaN,NaN
4,04-05-2023,0.032900,NaN,NaN
...,...,...,...,...
66,06-06-2023,0.036500,0.0388,0.0374
67,06-07-2023,0.036600,0.0385,0.0374
68,06-08-2023,0.036500,0.0386,0.0374
69,06-09-2023,0.035800,0.0384,0.0374


In [180]:
updated_df['on_demand'] = '0.0416'

In [181]:
updated_df['ds'] = pd.to_datetime(updated_df['ds'], format='%m-%d-%Y')
updated_df['ds'] = updated_df['ds'].dt.strftime('%m-%d')

In [182]:
updated_df

,ds,y,yhat,bidding_price,on_demand
0,04-01,0.027900,NaN,NaN,0.0672
1,04-02,0.029100,NaN,NaN,0.0672
2,04-03,0.031100,NaN,NaN,0.0672
3,04-04,0.032400,NaN,NaN,0.0672
4,04-05,0.032900,NaN,NaN,0.0672
...,...,...,...,...,...
66,06-06,0.036500,0.0388,0.0374,0.0672
67,06-07,0.036600,0.0385,0.0374,0.0672
68,06-08,0.036500,0.0386,0.0374,0.0672
69,06-09,0.035800,0.0384,0.0374,0.0672


In [183]:
updated_df.to_csv(f'{instance_name}.csv', index=False)